#**PREPROCESAMIENTO Y NORMALIZACIÓN EN TEST BAJO TRAIN**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
## Cargar datos fd_val
fd_test = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test.csv", sep=';')

print(f"Registros en fd_test: {len(fd_test)}")

Registros en fd_test: 627


In [ ]:
fd_test.shape

(627, 72)

##Normalización y redimensionado de datos numéricos y categóricos DATOS TABULARES fd_test basado en TRAIN

In [ ]:

# Definir las columnas que no aportan información relevante
not_useful_columns = [
    'first_review', 'last_review', 'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms',
    'calculated_host_listings_count', 'license', 'calendar_last_scraped',
    'has_availability', 'availability_30', 'availability_60', 'availability_90',
    'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'host_listings_count',
    'host_neighbourhood', 'host_picture_url', 'host_about', 'host_location',
    'host_thumbnail_url', 'host_name', 'host_url', 'host_id', 'source',
    'last_scraped', 'scrape_id', 'neighbourhood_group_cleansed', 'calendar_updated'
]

# Hacemos un drop de todas las columnas que no aportan información relevante a precio o inmueble en fd_test
fd_test.drop(not_useful_columns, axis=1, inplace=True)

fd_test.shape

(627, 41)

##nulls en review_scores_* y reviews_per_month

In [ ]:
import pandas as pd
import joblib

# Definir las columnas a imputar
review_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                  'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                  'review_scores_value', 'reviews_per_month']

# Cargar las medianas previamente guardadas del archivo
medianas_reviews = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/medianas_reviews.pkl')

# Imputar valores nulos en el conjunto de prueba con las medianas
for column in review_columns:
    fd_test[column] = fd_test[column].fillna(medianas_reviews[column])

# Imprimir el número de nulos restantes después de la imputación
print("Número de nulos restantes después de la imputación en test:")
print(fd_test[review_columns].isnull().sum())


Número de nulos restantes después de la imputación en test:
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
reviews_per_month              0
dtype: int64


##Imputacion 'bathrooms' y 'bathrooms_text'

In [ ]:
import re
import pandas as pd
import joblib

# Cargar la información de imputación previamente guardada
bathrooms_info = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/bathrooms_info.pkl')
bathrooms_median = bathrooms_info['bathrooms_median']
porcentaje_bathrooms_shared = bathrooms_info['porcentaje_bathrooms_shared']

# Extraemos el número de baños de bathrooms_text
def extract_bathrooms(text):
    if pd.isnull(text):
        return None
    match = re.search(r'(\d+)', text)
    return float(match.group(1)) if match else None

# Crear una nueva columna 'bathrooms_shared' para indicar si el baño es compartido o no
fd_test['bathrooms_shared'] = fd_test['bathrooms_text'].apply(lambda x: 1 if 'shared' in str(x).lower() else 0)

# Imputar bathrooms usando bathrooms_text cuando bathrooms está en nulo
fd_test['bathrooms'] = fd_test['bathrooms'].combine_first(fd_test['bathrooms_text'].apply(extract_bathrooms))

# Imputar valores faltantes restantes con la mediana de 'bathrooms' calculada del train
fd_test['bathrooms'] = fd_test['bathrooms'].fillna(bathrooms_median)

# Imputar valores faltantes restantes en 'bathrooms_shared' usando el porcentaje calculado
fd_test['bathrooms_shared'] = fd_test['bathrooms_shared'].fillna(round(porcentaje_bathrooms_shared))

# Eliminar bathrooms_text si ya no tiene nulos en bathrooms y bathrooms_shared
if fd_test['bathrooms'].isnull().sum() == 0 and fd_test['bathrooms_shared'].isnull().sum() == 0:
    fd_test.drop(columns=['bathrooms_text'], inplace=True)
    print("'bathrooms_text' ha sido eliminada porque ya no hay nulos en 'bathrooms' y 'bathrooms_shared'.")
else:
    print("'bathrooms_text' no ha sido eliminada porque aún hay nulos en 'bathrooms' o 'bathrooms_shared'.")

# Imprimir el número de nulos restantes después de la imputación
print("Nulos en 'bathrooms' después de la imputación en test:", fd_test['bathrooms'].isnull().sum())
print("Nulos en 'bathrooms_shared' en test:", fd_test['bathrooms_shared'].isnull().sum())

'bathrooms_text' ha sido eliminada porque ya no hay nulos en 'bathrooms' y 'bathrooms_shared'.
Nulos en 'bathrooms' después de la imputación en test: 0
Nulos en 'bathrooms_shared' en test: 0


In [ ]:
fd_test.shape

(627, 41)

# Imputar 'beds'

In [ ]:
import pandas as pd
import joblib

# Definir las columnas a imputar
group_columns = ['property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_shared']

# Cargar las medianas previamente guardadas del archivo
median_beds_grouped = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/median_beds_grouped.pkl')
mediana_global_beds = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/mediana_global_beds.pkl')

# Imputar valores nulos en el conjunto de prueba con las medianas calculadas por grupo
fd_test = fd_test.merge(median_beds_grouped, on=group_columns, how='left')
fd_test['beds'] = fd_test['beds'].fillna(fd_test['beds_median_grouped'])

# Imputar cualquier valor restante con la mediana global de 'beds'
fd_test['beds'] = fd_test['beds'].fillna(mediana_global_beds)

# Eliminar la columna temporal 'beds_median_grouped' después de la imputación
fd_test.drop(columns=['beds_median_grouped'], inplace=True)

# Imprimir el número de nulos restantes después de la imputación
print("Nulos restantes en 'beds' después de la imputación en test:", fd_test['beds'].isnull().sum())

Nulos restantes en 'beds' después de la imputación en test: 0


# Imputar 'bedrooms'


In [ ]:
import numpy as np
import pandas as pd
from difflib import get_close_matches

# Definir los valores conocidos de 'room_type'
known_room_types = ["Entire home/apt", "Private room", "Hotel room", "Shared room"]

# Función de imputación de 'bedrooms' basada en reglas mejoradas (misma que en train)
def imputar_bedrooms(df):
    for index, row in df.iterrows():
        room_type = row['room_type']

        # Normalizar valor: Convertir a minúsculas para evitar problemas de mayúsculas/minúsculas
        room_type_normalized = room_type.lower() if pd.notnull(room_type) else None

        # Intentar encontrar coincidencias aproximadas con los valores conocidos
        match = get_close_matches(room_type_normalized, [rt.lower() for rt in known_room_types], n=1, cutoff=0.8)

        # Si encontramos una coincidencia, utilizamos el valor conocido
        if match:
            room_type_final = match[0].capitalize()
        else:
            # Si no hay coincidencias, intentamos hacer una clasificación basada en accommodates
            accommodates = row['accommodates']
            if pd.notnull(accommodates):
                if accommodates <= 2:
                    room_type_final = "Private room"  # Suposición razonable si solo tiene capacidad para 2
                elif accommodates >= 3 and accommodates <= 6:
                    room_type_final = "Entire home/apt"  # Suposición razonable para acomodar 3 a 6 personas
                else:
                    room_type_final = "Entire home/apt"  # Suposición razonable para más de 6 personas
            else:
                # Si no podemos hacer una clasificación basada en accommodates, asignamos un valor por defecto
                room_type_final = "Unknown"

        # Aplicar reglas para imputar 'bedrooms'
        if room_type_final == "Entire home/apt":
            # Reglas para 'Entire home/apt'
            if row['accommodates'] <= 2:
                df.at[index, 'bedrooms'] = 1
            elif 3 <= row['accommodates'] <= 4:
                df.at[index, 'bedrooms'] = 2
            elif 5 <= row['accommodates'] <= 6:
                df.at[index, 'bedrooms'] = 3
            else:
                df.at[index, 'bedrooms'] = np.ceil(row['accommodates'] / 2)

        elif room_type_final == "Private room":
            # Regla para 'Private room'
            df.at[index, 'bedrooms'] = 1

        elif room_type_final in ["Hotel room", "Shared room"]:
            # Regla para 'Hotel room' y 'Shared room'
            df.at[index, 'bedrooms'] = 1

        elif room_type_final == "Unknown":
            # Valor predeterminado para un 'room_type' desconocido, basándose en el número de accommodates
            df.at[index, 'bedrooms'] = np.ceil(row['accommodates'] / 2) if pd.notnull(row['accommodates']) else 1

    return df

# Aplicar la función a fd_test (misma lógica que en val y train)
fd_test = imputar_bedrooms(fd_test)

# Verificar nulos restantes en 'bedrooms' después de la imputación en test
print("Nulos restantes en 'bedrooms' después de la imputación en fd_test:", fd_test['bedrooms'].isnull().sum())

Nulos restantes en 'bedrooms' después de la imputación en fd_test: 0


In [ ]:
# Identificar columnas que contienen la palabra 'host' en su nombre
host_columns = [col for col in fd_test.columns if 'host' in col]

# Mostrar valores nulos en estas columnas
null_counts = fd_test[host_columns].isnull().sum()

print("Valores nulos en las columnas que contienen 'host':")
print(null_counts)


Valores nulos en las columnas que contienen 'host':
host_since                    0
host_response_time           51
host_response_rate           51
host_acceptance_rate         38
host_is_superhost             7
host_total_listings_count     0
host_verifications            0
host_has_profile_pic          0
host_identity_verified        0
dtype: int64


###Imputacion de NULLS columnas catégoricas:
Imputar columnas host_

In [ ]:
import joblib
import numpy as np
import pandas as pd

# Cargar la información guardada de train
host_info = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/host_info.pkl')
mean_host_response_rate = host_info['mean_host_response_rate']
mean_host_acceptance_rate = host_info['mean_host_acceptance_rate']
mode_host_is_superhost = host_info['mode_host_is_superhost']

# Imputar 'host_is_superhost' con el valor más frecuente (moda) en el dataset
fd_test['host_is_superhost'] = fd_test['host_is_superhost'].fillna(mode_host_is_superhost)

# Convertir 'host_response_rate' y 'host_acceptance_rate' de texto a valores numéricos quitando el % y dividiendo el número por 100
if fd_test['host_response_rate'].dtype == 'O':
    fd_test['host_response_rate'] = fd_test['host_response_rate'].str.rstrip('%').astype(float) / 100
if fd_test['host_acceptance_rate'].dtype == 'O':
    fd_test['host_acceptance_rate'] = fd_test['host_acceptance_rate'].str.rstrip('%').astype(float) / 100

# Imputar 'host_response_rate' y 'host_acceptance_rate' basado en la media dentro de los grupos de 'host_is_superhost'
fd_test['host_response_rate'] = fd_test.apply(
    lambda row: mean_host_response_rate.get(row['host_is_superhost'], np.nan) if pd.isnull(row['host_response_rate']) else row['host_response_rate'], axis=1)
fd_test['host_acceptance_rate'] = fd_test.apply(
    lambda row: mean_host_acceptance_rate.get(row['host_is_superhost'], np.nan) if pd.isnull(row['host_acceptance_rate']) else row['host_acceptance_rate'], axis=1)

# Imputar 'host_response_time' basado en reglas condicionales en función de 'host_is_superhost' y 'host_response_rate'
def imputar_host_response_time(row):
    if pd.isnull(row['host_response_time']):
        if row['host_is_superhost'] == 't' or (row['host_response_rate'] is not None and row['host_response_rate'] >= 0.9):
            return 'within an hour'
        elif row['host_response_rate'] is not None and 0.5 <= row['host_response_rate'] < 0.9:
            return 'within a day'
        else:
            return 'a few days or more'
    return row['host_response_time']

# Aplicar la imputación condicional a 'host_response_time'
fd_test['host_response_time'] = fd_test.apply(imputar_host_response_time, axis=1)

# Verificar nulos restantes en las columnas host_*
print("Nulos restantes en host_response_time (test):", fd_test['host_response_time'].isnull().sum())
print("Nulos restantes en host_response_rate (test):", fd_test['host_response_rate'].isnull().sum())
print("Nulos restantes en host_acceptance_rate (test):", fd_test['host_acceptance_rate'].isnull().sum())
print("Nulos restantes en host_is_superhost (test):", fd_test['host_is_superhost'].isnull().sum())

Nulos restantes en host_response_time (test): 0
Nulos restantes en host_response_rate (test): 0
Nulos restantes en host_acceptance_rate (test): 0
Nulos restantes en host_is_superhost (test): 0


##NULLS 'neighbourhood', primero CODIFICACIÓN

In [ ]:
# Verificar las categorías únicas en 'neighbourhood_cleansed'
unique_neighbourhood_cleansed = fd_test['neighbourhood_cleansed'].unique()
print(unique_neighbourhood_cleansed)
print(f'Número total de categorías únicas en neighbourhood_cleansed: {len(unique_neighbourhood_cleansed)}')

['Fenway' 'South Boston Waterfront' 'South End' 'Downtown' 'East Boston'
 'South Boston' 'Charlestown' 'North End' 'Roxbury' 'Brighton'
 'Dorchester' 'Bay Village' 'Beacon Hill' 'Back Bay' 'Jamaica Plain'
 'Roslindale' 'West Roxbury' 'Allston' 'Hyde Park' 'Mattapan'
 'Longwood Medical Area' 'Leather District' 'Mission Hill' 'Chinatown'
 'West End']
Número total de categorías únicas en neighbourhood_cleansed: 25


In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder

# Cargar el LabelEncoder y los centroides previamente guardados
le_combined = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/label_encoder_combined.pkl")
centroides = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/centroides_neighbourhood.pkl")

# Definición de la función haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))


# Limpieza y estandarización de las columnas 'neighbourhood' y 'neighbourhood_cleansed' en prueba
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.strip()
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.replace(r"\s+", " ", regex=True)
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.replace(r", ,", ",", regex=True)
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.replace(r"Dorchster", "Dorchester", regex=False)
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.replace(r"\s+,", ",", regex=True)
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.replace(r"\(.*\)", "", regex=True).str.strip()
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.replace(r", Massachusetts, United States", "", regex=False)
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.strip()
fd_test['neighbourhood'] = fd_test['neighbourhood'].str.replace(r"Jamaica Plain, Boston", "Jamaica Plain", regex=False)

fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.strip()
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.replace(r"\s+", " ", regex=True)
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.replace(r", ,", ",", regex=True)
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.replace(r"Dorchster", "Dorchester", regex=False)
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.replace(r"\s+,", ",", regex=True)
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.replace(r"\(.*\)", "", regex=True).str.strip()
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.replace(r", Massachusetts, United States", "", regex=False)
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].str.strip()

# Manejar valores desconocidos para 'neighbourhood' usando los centroides
def imputar_barrio_mas_cercano(row, centroides):
    if row['neighbourhood'] not in le_combined.classes_:
        # Calcular distancias geográficas a los centroides y asignar el barrio más cercano
        distancias = centroides.apply(lambda x: haversine(row['latitude'], row['longitude'], x['latitude'], x['longitude']), axis=1)
        barrio_mas_cercano = distancias.idxmin()
        return barrio_mas_cercano
    return row['neighbourhood']

# Aplicar la imputación para barrios desconocidos
fd_test['neighbourhood'] = fd_test.apply(imputar_barrio_mas_cercano, axis=1, args=(centroides,))

# Codificar las columnas utilizando el LabelEncoder
fd_test['neighbourhood'] = fd_test['neighbourhood'].apply(
    lambda x: le_combined.transform([x])[0] if x in le_combined.classes_ else le_combined.transform(["<UNK>"])[0]
)
fd_test['neighbourhood_cleansed'] = fd_test['neighbourhood_cleansed'].apply(
    lambda x: le_combined.transform([x])[0] if x in le_combined.classes_ else le_combined.transform(["<UNK>"])[0]
)

print("Valores únicos resultantes de 'neighbourhood' y 'neighbourhood_cleansed' en test:")
print(fd_test['neighbourhood'].unique())
print(fd_test['neighbourhood_cleansed'].unique())


Valores únicos resultantes de 'neighbourhood' y 'neighbourhood_cleansed' en test:
[ 5 12  6 10  7 15 14 21  9 27]
[13 24 25 11 12 23  7 20 22  6 10  3  4  2 15 21 27  1 14 18 17 16 19  8
 26]


##Imputación 'neighbourhood'

In [ ]:
from sklearn.impute import KNNImputer

import joblib

# Cargar el imputador KNN previamente ajustado en el conjunto de entrenamiento
imputer = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/knn_imputer_trained.pkl")

# Definir las columnas que se utilizarán para la imputación
columns_for_imputation = ['latitude', 'longitude', 'neighbourhood', 'neighbourhood_cleansed']

# Aplicar el imputador al conjunto de prueba
fd_test_imputed = imputer.transform(fd_test[columns_for_imputation])

# Actualizar el DataFrame con los valores imputados en 'neighbourhood'
fd_test['neighbourhood'] = fd_test_imputed[:, 2]

# Guardar el dataset de prueba imputado si es necesario
fd_test.to_csv('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_imputed.csv', sep=';', index=False)

In [ ]:
# Identificar columnas categóricas con valores nulos
categorical_columns = fd_test.select_dtypes(include=['object']).columns
categorical_null_columns = categorical_columns[fd_test[categorical_columns].isnull().any()]

# Contar los valores nulos en cada una de las columnas categóricas con nulos
categorical_null_counts = fd_test[categorical_null_columns].isnull().sum()
print(categorical_null_counts)


Series([], dtype: float64)


In [ ]:
fd_test.shape

(627, 41)

##Ya no tenemos NULLS ahora convertiremos las categoricas a numericas restantes:

In [ ]:
#cargar dataset fd_test_imputed
fd_test_imputed = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_imputed.csv", sep=';')

##Comenzaré con las categorias host_

In [ ]:
from datetime import datetime

fd_test_imputed['host_since'] = fd_test_imputed['host_since'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
fd_test_imputed['years_being_host'] = fd_test_imputed['host_since'].apply(lambda x: datetime.now().year - x.year)

fd_test_imputed.drop(['host_since'], axis=1, inplace=True)

print(fd_test_imputed['years_being_host'].dtype)

int64


##'host_response_time' valores unicos

In [ ]:
# Mapping para convertir los valores de 'host_response_time' en valores numéricos relevancia
response_time_mapping = {
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1
}

# Aplicar la transformación a la columna 'host_response_time'
fd_test_imputed['host_response_time'] = fd_test_imputed['host_response_time'].map(response_time_mapping)

# Verificar los valores únicos después de la transformación
print(fd_test_imputed['host_response_time'].unique())

[4 3 1 2]


##'host_is_superhost' ,'host_has_profile_pic', 'host_identity_verified' e 'instant_bookable'

In [ ]:
# Convertir las cadenas 't' y 'f' a valores numéricos binarios (0 y 1) usando .apply()

boolean_columns = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
# Convertir las cadenas 't' y 'f' a valores numéricos binarios (0 y 1) usando .apply()
for col in boolean_columns:
    fd_test_imputed[col] = fd_test_imputed[col].apply(lambda x: 1 if x == 't' else 0)

# Verificar los valores únicos para confirmar la transformación
for col in boolean_columns:
    print(f"Valores únicos en {col} (test):", fd_test_imputed[col].unique())


Valores únicos en host_is_superhost (test): [1 0]
Valores únicos en host_has_profile_pic (test): [1 0]
Valores únicos en host_identity_verified (test): [1 0]
Valores únicos en instant_bookable (test): [0 1]


##'host_verifications'

In [ ]:
print("Valores únicos en 'host_verifications':")
print(fd_test_imputed['host_verifications'].unique())


Valores únicos en 'host_verifications':
["['email', 'phone']" "['email', 'phone', 'work_email']" "['phone']"
 "['phone', 'work_email']" '[]']


In [ ]:
from ast import literal_eval
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargar el escalador guardado desde train
scaler = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_verifications.pkl')

# Convertir las cadenas de texto en listas reales usando literal_eval
fd_test_imputed['host_verifications'] = fd_test_imputed['host_verifications'].apply(literal_eval)

# Normalizar las listas eliminando espacios innecesarios y asegurando un formato consistente
fd_test_imputed['host_verifications'] = fd_test_imputed['host_verifications'].apply(
    lambda x: [item.strip().lower() for item in x]
)

# Contar el número de verificaciones para cada anfitrión, asegurando que las listas vacías sean tratadas como 0
fd_test_imputed['num_host_verifications'] = fd_test_imputed['host_verifications'].apply(
    lambda x: len(x) if len(x) > 0 else 0  # Listas vacías -> 0
)

# Clipping en el rango esperado antes del escalado
fd_test_imputed['num_host_verifications'] = np.clip(
    fd_test_imputed['num_host_verifications'], scaler.data_min_[0], scaler.data_max_[0]
)

# Escalar los valores usando el escalador de train
fd_test_imputed['num_host_verifications'] = scaler.transform(
    fd_test_imputed[['num_host_verifications']]
)

# Eliminar la columna original 'host_verifications' para evitar redundancia
fd_test_imputed.drop('host_verifications', axis=1, inplace=True)

# Verificar las primeras filas para asegurarnos de que la transformación se realizó correctamente
print(fd_test_imputed[['num_host_verifications']].agg(['min', 'max']))


     num_host_verifications
min                    0.01
max                    0.99


In [ ]:
# Ver las categorías únicas de 'num_host_verifications' y su frecuencia
unique_categories = fd_test_imputed['num_host_verifications'].value_counts()

print("Categorías únicas en 'num_host_verifications' y sus frecuencias:")
print(unique_categories)

# Si solo quieres los valores únicos sin la frecuencia:
unique_values = fd_test_imputed['num_host_verifications'].unique()
print("\nValores únicos en 'num_host_verifications':")
print(unique_values)


Categorías únicas en 'num_host_verifications' y sus frecuencias:
num_host_verifications
0.50    499
0.99     68
0.01     60
Name: count, dtype: int64

Valores únicos en 'num_host_verifications':
[0.5  0.99 0.01]


In [ ]:
# Verificar mínimo y máximo de la columna 'num_host_verifications' en fd_test_finpreprocesado
min_max_verifications = fd_test_imputed['num_host_verifications'].agg(['min', 'max'])
print("Mínimo y máximo de 'num_host_verifications' en fd_test_imputedo:")
print(min_max_verifications)

Mínimo y máximo de 'num_host_verifications' en fd_test_imputedo:
min    0.01
max    0.99
Name: num_host_verifications, dtype: float64


In [ ]:
fd_test_imputed.shape

(627, 41)

##'property_type'

In [ ]:
import joblib

# Cargar las medias calculadas para 'property_type' desde el conjunto de entrenamiento
property_price_means = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/property_price_means.pkl")

# Crear una nueva columna 'property_type_encoded' en el conjunto de prueba
fd_test_imputed['property_type_encoded'] = fd_test_imputed['property_type'].map(property_price_means)

# Manejar valores no vistos en el diccionario, asignándoles la media global calculada en el conjunto de entrenamiento
global_mean_price_train = sum(property_price_means.values()) / len(property_price_means)
fd_test_imputed['property_type_encoded'] = fd_test_imputed['property_type_encoded'].fillna(global_mean_price_train)

# Eliminar la columna original 'property_type'
fd_test_imputed.drop('property_type', axis=1, inplace=True)

# Verificar las primeras filas para asegurarnos de que la transformación se realizó correctamente
print(fd_test_imputed[['property_type_encoded']].head())


   property_type_encoded
0             190.545852
1             224.561404
2             190.545852
3             190.545852
4             190.545852


##'room_type'

In [ ]:
import joblib

# Cargar las medias calculadas previamente y la media global desde el entrenamiento
room_data = joblib.load("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/room_price_means.pkl")
room_price_means = room_data['room_price_means']
global_mean_price_train = room_data['global_mean_price_train']

# Aplicar el promedio calculado en train para codificar la columna en test
fd_test_imputed['room_type_encoded'] = fd_test_imputed['room_type'].map(room_price_means)

# Manejar valores no vistos en train asignándoles la media global
fd_test_imputed['room_type_encoded'] = fd_test_imputed['room_type_encoded'].fillna(global_mean_price_train)

# Eliminar la columna original 'room_type' para evitar redundancia
fd_test_imputed.drop('room_type', axis=1, inplace=True)

# Verificar las primeras filas para asegurarnos de que la transformación se realizó correctamente
print(fd_test_imputed[['room_type_encoded']].head())

   room_type_encoded
0         217.807606
1         217.807606
2         217.807606
3         217.807606
4         217.807606


###Eliminamos columnas que finalmente son innecesarias

In [ ]:
# Eliminar columnas innecesarias del conjunto test
fd_test_imputed.drop(columns=['listing_url', 'picture_url', 'amenities', 'id'], inplace=True)

# Verificar que las columnas han sido eliminadas
print("Columnas restantes en fd_test_imputed:", fd_test.columns)

Columnas restantes en fd_test_imputed: Index(['id', 'listing_url', 'picture_url', 'host_since', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month',
       'bathrooms_shared'],
      dtype='object')


In [ ]:
fd_test_imputed.shape

(627, 37)

In [ ]:
# Verificar si existen columnas no numéricas en fd_train_imputed
non_numeric_columns = fd_test_imputed.select_dtypes(exclude=['number']).columns

# Imprimir las columnas no numéricas
print("Columnas no numéricas en fd_test_imputed:", non_numeric_columns)


Columnas no numéricas en fd_test_imputed: Index([], dtype='object')


In [ ]:
# Guardar el conjunto test imputado y con todas las columnas convertidas a numéricas
fd_test_imputed.to_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_imputed_num.csv", sep=';', index=False)

# Confirmación del guardado
print("El archivo CSV con los datos imputados y numéricos ha sido guardado como 'fd_test_imputed_num.csv'")

El archivo CSV con los datos imputados y numéricos ha sido guardado como 'fd_test_imputed_num.csv'


In [ ]:
# Cargar el conjunto test imputado y numérico desde el archivo CSV
fd_test_imputed_num = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_imputed_num.csv", sep=';')

# Confirmación de la carga
print("El archivo CSV 'fd_test_imputed_num.csv' ha sido cargado exitosamente.")

El archivo CSV 'fd_test_imputed_num.csv' ha sido cargado exitosamente.


##OUTLIERS
"price"

In [ ]:
# Número de registros antes de eliminar outliers inconsistentes en test
initial_count_test = fd_test_imputed_num.shape[0]

# Definir un umbral para el precio extremo basado en entrenamiento
price_threshold = 3000

# Filtrar registros inconsistentes con precio mayor a $3,000 y características poco justificadas en test
inconsistent_outliers_test = fd_test_imputed_num[
    (fd_test_imputed_num['price'] > price_threshold) &
    (fd_test_imputed_num['accommodates'] <= 2) &
    (fd_test_imputed_num['beds'] <= 2)
]

# Número de registros inconsistentes que se eliminarán en test
outliers_count_test = inconsistent_outliers_test.shape[0]

# Eliminar esos registros inconsistentes del dataset de test
fd_test_imputed_num = fd_test_imputed_num.drop(inconsistent_outliers_test.index)

# Número de registros restantes después de eliminar los outliers en test
final_count_test = fd_test_imputed_num.shape[0]

# Mostrar cuántos registros se eliminaron en test
print(f"Registros eliminados en test: {outliers_count_test}")
print(f"Registros restantes después de eliminar outliers inconsistentes en test: {final_count_test}")

Registros eliminados en test: 0
Registros restantes después de eliminar outliers inconsistentes en test: 627


##Transformación logarítmica en 'price'

In [ ]:
import numpy as np

# Aplicar la transformación logarítmica con `np.log1p` a la columna 'price'
fd_test_imputed_num['price_log'] = np.log1p(fd_test_imputed_num['price'])

# Verificar la transformación
print(fd_test_imputed_num[['price', 'price_log']].head())

# Eliminar la columna original 'price' para evitar problemas de colinealidad
fd_test_imputed_num.drop(columns=['price'], inplace=True)

   price  price_log
0  149.0   5.010635
1  200.0   5.303305
2  152.0   5.030438
3  109.0   4.700480
4  153.0   5.036953


Capping a 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_reviews', 'reviews_per_month', 'host_total_listings_count', 'minimum_nights', 'maximum_nights'

In [ ]:
import numpy as np
import pickle

# Cargar los thresholds calculados previamente en train
with open('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/capping_thresholds.pkl', 'rb') as f:
    capping_thresholds = pickle.load(f)

# Aplicar capping a las columnas en el conjunto de prueba
for column in ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_reviews', 'reviews_per_month', 'host_total_listings_count', 'minimum_nights', 'maximum_nights']:
    # Obtener el threshold para la columna
    threshold = capping_thresholds[column]

    # Aplicar capping en los valores que superan el threshold
    fd_test_imputed_num[column] = np.where(fd_test_imputed_num[column] > threshold, threshold, fd_test_imputed_num[column])

# Verificar algunos valores después del capping en fd_test
print("Valores después del capping en 'host_total_listings_count' (test):")
print(fd_test_imputed_num['host_total_listings_count'].sort_values(ascending=False).head())

Valores después del capping en 'host_total_listings_count' (test):
613    1436.0
510    1436.0
449    1436.0
274    1436.0
164    1436.0
Name: host_total_listings_count, dtype: float64


In [ ]:
# Guardar el fd_test_imputed_num_out preprocesado CON TRABAJO DE OUTLIERS en un archivo CSV
fd_test_imputed_num.to_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_imputed_num_out.csv", sep=';', index=False)

In [ ]:
# Cargar el fd preprocesado preprocesado CON TRABAJO DE OUTLIERS desde un archivo CSV
import pandas as pd
fd_test_imputed_num_out = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_imputed_num_out.csv", sep=';')

In [ ]:
fd_test_imputed_num_out.shape

(627, 37)

Eliminar 'neighbourhood por que tiene mas correlacion con 'price_log 'neighbourhood_cleansed'

In [ ]:
#drop de 'neighbourhood'
fd_test_imputed_num_out.drop(columns=['neighbourhood'], inplace=True)

In [ ]:
fd_test_imputed_num_out.shape

(627, 36)

#NORMALIZACIÓN

####Normalizare 'bedrooms', 'beds', 'accommodates', 'bathrooms', que tienen valores como 1, 0 [0.01, 0.99]. Para que mas adelante no presenten problemas con funciones de activacion como sigmoid o tanh.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargar el escalador entrenado en train
scaler_bedrooms_beds_adjusted = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_bedrooms_beds_adjusted.pkl')

# Aplicar el escalador a 'bedrooms', 'beds', 'accommodates', y 'bathrooms' en prueba
fd_test_imputed_num_out[['bedrooms', 'beds', 'accommodates', 'bathrooms']] = scaler_bedrooms_beds_adjusted.transform(
    fd_test_imputed_num_out[['bedrooms', 'beds', 'accommodates', 'bathrooms']]
)

# Verificar los valores después del ajuste
adjusted_bedrooms_beds_stats_test = fd_test_imputed_num_out[['bedrooms', 'beds', 'accommodates', 'bathrooms']].agg(['min', 'max'])

# Mostrar los resultados
print(adjusted_bedrooms_beds_stats_test)

     bedrooms  beds  accommodates  bathrooms
min      0.01  0.01          0.01       0.01
max      0.99  0.99          0.99       0.99


#Columnas Numéricas para Normalización

In [ ]:
import numpy as np
import joblib

# Cargar configuraciones guardadas de train
columns_log_transform = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/log_transform_columns.pkl')
scaler = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_log_transform.pkl')

# Revisar valores antes de cualquier corrección
print("Valores originales en Test (test):")
print(fd_test_imputed_num_out[columns_log_transform].agg(['min', 'max']))

# Aplicar log1p (sin valores negativos)
fd_test_imputed_num_out[columns_log_transform] = fd_test_imputed_num_out[columns_log_transform].clip(lower=0).apply(np.log1p)

# Revisar valores después de log1p
print("Valores después de log1p en Test (test):")
print(fd_test_imputed_num_out[columns_log_transform].agg(['min', 'max']))

# Asegurar que los valores estén dentro del rango observado en train
for col in columns_log_transform:
    min_train = scaler.data_min_[columns_log_transform.index(col)]
    max_train = scaler.data_max_[columns_log_transform.index(col)]
    # Clipping a los rangos observados en train
    fd_test_imputed_num_out[col] = fd_test_imputed_num_out[col].clip(lower=min_train, upper=max_train)

# Aplicar el escalador para normalizar los valores
fd_test_imputed_num_out[columns_log_transform] = scaler.transform(fd_test_imputed_num_out[columns_log_transform])

# Verificar los valores finales después del escalado
print("Valores finales en Test (test):")
print(fd_test_imputed_num_out[columns_log_transform].agg(['min', 'max']))


Valores originales en Test (test):
     host_total_listings_count  minimum_nights  maximum_nights  \
min                        1.0             1.0             3.0   
max                     1436.0           115.2          1125.0   

     minimum_minimum_nights  availability_365  number_of_reviews  \
min                       1                 0               0.00   
max                     180               365             375.52   

     number_of_reviews_ltm  number_of_reviews_l30d  reviews_per_month  \
min                      0                       0             0.0100   
max                    153                      10             8.9256   

     property_type_encoded  room_type_encoded  
min              38.000000          51.833333  
max             677.809524         508.357143  
Valores después de log1p en Test (test):
     host_total_listings_count  minimum_nights  maximum_nights  \
min                   0.693147        0.693147        1.386294   
max                   7.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargar el escalador y las columnas seleccionadas desde train
scaler_min_max = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/scaler_min_max_train.pkl')
columns_min_max = joblib.load('/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/columns_min_max.pkl')

# Aplicar el escalador a las columnas seleccionadas
fd_test_imputed_num_out[columns_min_max] = scaler_min_max.transform(fd_test_imputed_num_out[columns_min_max])

# Verificar las estadísticas después del escalado
print("Estadísticas después del escalado en prueba (test):")
print(fd_test_imputed_num_out[columns_min_max].agg(['min', 'max']))

Estadísticas después del escalado en prueba (test):
     host_response_rate  host_acceptance_rate  latitude  longitude  \
min                0.01                  0.01  0.009812   0.015698   
max                0.99                  0.99  0.983673   0.958077   

     review_scores_rating  review_scores_accuracy  review_scores_cleanliness  \
min                  0.01                   0.206                      0.206   
max                  0.99                   0.990                      0.990   

     review_scores_checkin  review_scores_communication  \
min                  0.206                         0.01   
max                  0.990                         0.99   

     review_scores_location  review_scores_value  years_being_host  
min                    0.01                 0.01              0.01  
max                    0.99                 0.99              1.06  


###*'neighbourhood_cleansed'* ya paso por labelencoder y esta codficada.

No necesita en este momento normalizacion, puede trabajarse mas segun el modelo que haga si es con redes neuronales o arboles, etc.

In [ ]:
# Guardar el fd_test_imputed_num_out con final de preprocesado y NORMALIZACION en un archivo CSV fd_test_finpreprocesado.
fd_test_imputed_num_out.to_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_finpreprocesado.csv", sep=';', index=False)


In [ ]:
# Cargar el fd_test_finpreprocesado.csv
import pandas as pd
fd_test_finpreprocesado = pd.read_csv("/content/drive/MyDrive/Deep_Learning_2024/Notebooks/data/fd_test_finpreprocesado.csv", sep=';')

In [ ]:
fd_test_finpreprocesado.shape

(627, 36)

In [ ]:
import pandas as pd

# Configuración para mostrar todas las columnas sin truncar
pd.set_option('display.max_columns', None)

# Para fd_val_finalpreprocesado
print("\nMínimo y máximo en cada columna de fd_test_finpreprocesado:")
min_max_test = fd_test_finpreprocesado.agg(['min', 'max'])
print(min_max_test)

# Opcional: Restaurar configuración para evitar demasiadas columnas en futuras salidas
pd.reset_option('display.max_columns')


Mínimo y máximo en cada columna de fd_test_finpreprocesado:
     host_response_time  host_response_rate  host_acceptance_rate  \
min                   1                0.01                  0.01   
max                   4                0.99                  0.99   

     host_is_superhost  host_total_listings_count  host_has_profile_pic  \
min                  0                       0.01                     0   
max                  1                       0.99                     1   

     host_identity_verified  neighbourhood_cleansed  latitude  longitude  \
min                       0                       1  0.009812   0.015698   
max                       1                      27  0.983673   0.958077   

     accommodates  bathrooms  bedrooms  beds  minimum_nights  maximum_nights  \
min          0.01       0.01      0.01  0.01            0.01         0.05756   
max          0.99       0.99      0.99  0.99            0.99         0.99000   

     minimum_minimum_nights  availa